<a href="https://colab.research.google.com/github/82KJ/ARS-system-for-protecting-telephone-counselor/blob/main/SH/Test_1_to_5/sample_classifier_combination_test_3_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 기본 Setting

In [172]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [173]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [174]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [175]:
device = torch.device("cuda:0")

In [177]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /content/drive/MyDrive/종합설계/.cache/kobert_v1.zip
using cached model. /content/drive/MyDrive/종합설계/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


## 데이터셋 전처리 진행

In [178]:
import pandas as pd
data = pd.read_csv("training_dataset.csv")

In [179]:
data.head()

,Unnamed: 0,0,1
0,0,싯팔 호구샛끼야 닌 머가리 있냐? 눈 삠? 머리만 있어서 만만하냐?,1
1,1,여자들은 진짜 화장 하나에 얼굴 느낌이 확 변하네,0
2,2,여튼 조센징들은 옛날부터 언플과 선동에 너무나도 약했음.,1
3,3,선임으로 게이만나면 똥꼬 허벌창 되는거임,2
4,4,뷰지들 군대 보내봤자 오또케 오또케 이러고 있을 게 뻔한데.,2


In [180]:
print(len(data))

38693


In [181]:
len(data.loc[data['1'] == 0]),len(data.loc[data['1'] == 1]),len(data.loc[data['1'] == 2])

(20000, 10016, 8677)

In [182]:
data_list = list()
for sen, lab in zip(data["0"], data["1"]):
  data_list.append([sen,lab])

In [183]:
data_list[:10]

[['싯팔 호구샛끼야 닌 머가리 있냐? 눈 삠? 머리만 있어서 만만하냐?', 1],
 ['여자들은 진짜 화장 하나에 얼굴 느낌이 확 변하네', 0],
 ['여튼 조센징들은 옛날부터 언플과 선동에 너무나도 약했음.', 1],
 ['선임으로 게이만나면 똥꼬 허벌창 되는거임', 2],
 ['뷰지들 군대 보내봤자 오또케 오또케 이러고 있을 게 뻔한데.', 2],
 ['니가 안징징 거린다 했어 앞으로 징징거리면 다리 분질러 버린다 진짜 경고했다;', 1],
 ['걔네 응디 보면 애액이 줄줄 흐름 우웩', 2],
 ['저 군대가면 연예인 있다더라', 0],
 ['지금 뭐라고 했냐?', 0],
 ['그럼 당연하지!', 0]]

In [184]:
len(data_list)

38693

## Train data, Test data 분리

In [185]:
from sklearn.model_selection import train_test_split

In [193]:
train_set, test_set = train_test_split(data_list, test_size=0.1, random_state=0) # train : test = 8:2

In [194]:
len(train_set), len(test_set)

(34823, 3870)

In [196]:
train_set[0]

['그 동안 보빨러들이 얼마나 잘 해줬겠어ㅋㅋㅋ', 2]

## KoBERT 입력 데이터 구성하기
BERT 입력값은 (token + segment + position embedding의 합으로 구성)  
https://happy-obok.tistory.com/23 여기서 확인!!

In [197]:
class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):

    # sentence , label data를 BERT의 입력값에 맞게 변환하는 transformer를 생성
    transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_len, pad=pad, pair=pair)

    ## 생성한 transformer로 sentence를 변환하여 저장
    self.sentences = [transform([data[sent_idx]]) for data in dataset]
    self.labels = [np.int32(data[label_idx]) for data in dataset]
  
  def __getitem__ (self, i):
    return (self.sentences[i] + (self.labels[i], )) # 각 index에 맞는 item 반환 진행 --> 왜 이런 형태인지는 잘 모르겠음
  
  def __len__(self):
    return (len(self.labels))
    

In [198]:
# Parameter setting 진행
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [199]:
# Kobert 모듈에서 제공하는 get_tokenizer와 vocab를 활용해 tokneizer를 구성한다
tokenizer = get_tokenizer() 
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(train_set, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test_set, 0, 1, tok, max_len, True, False)

using cached model. /content/drive/MyDrive/종합설계/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [200]:
data_train[0]

## 출력값 설명
## 1번 array - 1로 padding된 sequence - 총 길이 64 고정
## 2번 array - padding 전 sequnce 길이 - 42
## 3번 array - 어텐션 마스크 sequence (1로 padding이 된 부분은 어텐션 함수가 적용되지 않아도 됨을 알려주는 시퀀스)
##           - 일단은 전부 0으로 구성하고, 아래의 gen_attention_mask에서 필요한 부분만 1로 변경
## 마지막 숫자 - label값

(array([   2, 1185, 1754, 2355, 6461, 6037, 5940, 3253, 3942, 4998, 7305,
        5405, 6855,  492,  492,  492,    3,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(17, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 2)

In [201]:
# Torch에 맞게 최종 입력 dataset 구성 

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## KoBert 다중 분류기를 위한 미세 조정 모델 만들기


In [202]:
class BERTClassifier(nn.Module):
  def __init__(self, bert, hidden_size = 768, num_classes=3, dr_rate=None, params=None):
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    ## classifier는 선형 회귀 모델로 구성 (input size = 768, output size = 4 (label이 4개로 구성))
    self.classifier = nn.Linear(hidden_size, num_classes)

    ## overfitting 방지를 위한 dropout 비율 설정
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)

  # attention mask sequence를 구성해주는 함수 --> padding이 아닌 영역을 0에서 1로 변경
  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i,v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    
    return attention_mask.float()
  
  # bert + classifier를 관통하는 forward 연산 진행
  def forward(self, token_ids, valid_length, segment_ids):

    # attention_mask 계산
    attention_mask = self.gen_attention_mask(token_ids, valid_length)

    # bert에 input 투입, 변수명이 pooler인거 보니 출력 embedding에 mean pooling 적용한 값이지 않을까 추측
    _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))

    # dropout 비율이 존재한다면, dropout 적용
    if self.dr_rate:
        out = self.dropout(pooler)

    # classifier 진행
    return self.classifier(out) 

In [203]:
# Kobert + classifier 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

# optimizer (Adam), scheduler 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

In [204]:
for e in range(num_epochs):

    train_acc = 0.0
    test_acc = 0.0
    model.train()

    # train set
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)

        # forward 연산 진행
        out = model(token_ids, valid_length, segment_ids)

        # loss 는 CrossEntropyLoss를 이용 -> backpropagation 학습 진행
        loss = loss_fn(out, label)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        # train data set 정확도 확인
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()

    # test set
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        
        # test set 정확도 확인
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-204-26ba33054b28>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/545 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.1746724843978882 train acc 0.265625
epoch 1 batch id 201 loss 0.5010502934455872 train acc 0.6138059701492538
epoch 1 batch id 401 loss 0.381879597902298 train acc 0.7260754364089775
epoch 1 train acc 0.7660509501965923


<ipython-input-204-26ba33054b28>:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/61 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9248804644808742


  0%|          | 0/545 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.0509490966796875 train acc 0.984375
epoch 5 batch id 201 loss 0.0801735669374466 train acc 0.9916044776119403
epoch 5 batch id 401 loss 0.0028277121018618345 train acc 0.9934149002493765
epoch 5 train acc 0.9938360091743119


  0%|          | 0/61 [00:00<?, ?it/s]

epoch 5 test acc 0.9253927595628415


## 학습된 Model로 결과 예측해보기

In [205]:
def predict(predict_sentence):

    # 1. data set 구성 (문장, 라벨)
    data = [predict_sentence, '0']
    dataset_another = [data]

    # 2. data를 bert의 입력에 맞게 변환하기
    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)

        # 모델 forward 연산 진행
        out = model(token_ids, valid_length, segment_ids)

        # torch out -> numpy 형식으로 변환
        test_eval=[]
        logits = out[0].detach().cpu().numpy()
        
        # 값이 가장 큰 인덱스를 출력
        if np.argmax(logits) == 0:
            test_eval= "일반"
        elif np.argmax(logits) == 1:
            test_eval = "폭언"
        elif np.argmax(logits) == 2:
            test_eval = "성희롱"
        # elif np.argmax(logits) == 3:
        #     test_eval = "성희롱"

        print(">> " + test_eval + " 문장입니다.")
        print(logits)
     

In [207]:
while True:
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "q":
      break
    
    predict(sentence)
    print()
     

하고싶은 말을 입력해주세요 : 안녕하세요 무엇을 도와드릴까요
>> 일반 문장입니다.
[ 5.485386  -3.0670972 -2.8355985]

하고싶은 말을 입력해주세요 : 시발 상담원년아 결제가 안돼잖아
>> 폭언 문장입니다.
[-2.3071754  5.1331024 -2.1155338]

하고싶은 말을 입력해주세요 : 일처리 좃같이 할래
>> 폭언 문장입니다.
[-2.3895495  4.854307  -1.8152132]

하고싶은 말을 입력해주세요 : 개새끼가 나한테 진짜 죽을래
>> 폭언 문장입니다.
[-2.1367614  5.115506  -2.3702579]

하고싶은 말을 입력해주세요 : 근데 너 목소리 섹시하다
>> 성희롱 문장입니다.
[ 1.265264  -4.043539   2.2824416]

하고싶은 말을 입력해주세요 : 오늘 나랑 모텔 가자
>> 성희롱 문장입니다.
[-2.5943773 -2.4303665  4.916949 ]

하고싶은 말을 입력해주세요 : q


In [208]:
torch.save(model.state_dict(),'kobert_classifier.pth')


In [209]:
model.load_state_dict(torch.load('kobert_classifier.pth'))

<All keys matched successfully>